In [2]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 71.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 25.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install hdfs

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for hdfs: filename=hdfs-2.7.3-py3-none-any.whl size=34431 sha256=c23f65b752d5cd1f81c3bbf3de1c8cb205797506c60695880d1f45f29d1d9c49
  Stored in directory: /home/student/.cache/pip/wheels/e5/8d/b6/99c1c0a3ac5788c866b0ecd3f48b0134a5910e6ed26011800b
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13781 sha256=fe8273bfc730e089e95837bdcf00cdd74f4f45f98897b2e4b6291089e931493e
  Stored in directory: /home/student/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built hdfs docopt
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install kafka-python

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 32.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dataingestion.tweet_fetcher import fetch_tweets
from dataingestion.twitter_api import get_user_data
from dataingestion.label_data import analyze_tweet_sentiment
from dataingestion.kafka_producer import send_tweets_to_kafka
from dataingestion.kafka_consumer import run_kafka_consumer
from dataingestion.store_to_HDFS import consume_tweets_to_hdfs


# get tweetID

In [5]:

# Get user input
username = input("Enter the username: ")

# Call the function
user_data = get_user_data(username)

# Check if we got valid data
if user_data[0] and user_data[1]:
    print(f"Username: {user_data[0]}, Rest ID: {user_data[1]}")
else:
    print("Failed to retrieve user data.")


Enter the username:  Malaysiakini


Querystring set to: {'username': 'Malaysiakini'}
Username: malaysiakini, Rest ID: 18040230


# Tweet fetcher

In [2]:
def main():
    # Define user IDs here in main
    user_ids = ["22594051", "55186601", "18040230", "61083422", "102098902", "145550026"]
    
    # Call the function with the user IDs
    tweets_data = fetch_tweets(
        user_ids=user_ids,
        count_per_user=50,
        output_file='tweets_output.csv'
    )
    
    # Example of how to use the returned data
    for user_id, tweets in tweets_data.items():
        print(f"User {user_id} has {len(tweets)} tweets")

# Fix the syntax in the main check
if __name__ == "__main__":
    main()

Fetched 50 tweets for user 22594051
Fetched 50 tweets for user 55186601
Fetched 50 tweets for user 18040230
Fetched 50 tweets for user 61083422
Fetched 50 tweets for user 102098902
Fetched 50 tweets for user 145550026

Successfully fetched data for all users!
Total tweets fetched: 300
User 22594051 has 50 tweets
User 55186601 has 50 tweets
User 18040230 has 50 tweets
User 61083422 has 50 tweets
User 102098902 has 50 tweets
User 145550026 has 50 tweets


# label data


In [3]:
analyzed_data = analyze_tweet_sentiment('tweets_output.csv')

Sentiment distribution:
Sentiment
Positive    125
Neutral      92
Negative     83
Name: count, dtype: int64

Sample of tweets with sentiment:
    User ID      Name  Followers Count  \
0  22594051  The Star          1903972   
1  22594051  The Star          1903972   
2  22594051  The Star          1903972   
3  22594051  The Star          1903972   
4  22594051  The Star          1903972   

                                               Tweet                Location  \
0  “There’s no scandal, no soap opera, no intrigue."  Kuala Lumpur, Malaysia   
1  This is following the implementation of the Wa...  Kuala Lumpur, Malaysia   
2  Defence Minister Ng Eng Hen said leadership su...  Kuala Lumpur, Malaysia   
3  Tony, Simon and Raymond's grandfather founded ...  Kuala Lumpur, Malaysia   
4  Since the US President slapped a 25% tariff on...  Kuala Lumpur, Malaysia   

                             Time  Friends Count Sentiment  
0  Fri Apr 18 08:35:13 +0000 2025            274  Negative  
1 

# kafka producer

In [4]:
from pathlib import Path
from dataingestion.kafka_producer import send_tweets_to_kafka
def main():
    """
    Main function to call the send_tweets_to_kafka function.
    """
    # Define parameters directly in the code
    csv_file_path = 'tweets_output_with_sentiment.csv'
    kafka_servers = ['localhost:9092']
    verbose = True
    
    # Validate CSV file path
    csv_path = Path(csv_file_path)
    if not csv_path.exists():
        print(f"Error: CSV file '{csv_file_path}' does not exist")
        return 1
    
    try:
        # Process the tweets
        tweet_count = send_tweets_to_kafka(
            csv_file=csv_file_path,
            bootstrap_servers=kafka_servers,
            verbose=verbose
        )
        
        print(f"Successfully processed {tweet_count} tweets")
        return 0
    except Exception as e:
        print(f"Error processing tweets: {e}")
        return 1

if __name__ == "__main__":
    main()

Tweet 1 data: {'user_id': 22594051, 'name': 'The Star', 'followers_count': 1903972, 'tweet_text': '“There’s no scandal, no soap opera, no intrigue."', 'location': 'Kuala Lumpur, Malaysia', 'created_at': 'Fri Apr 18 08:35:13 +0000 2025', 'friends_count': 274, 'sentiment': 'Negative'}
Sent to topic: MalaysiaNewsTopic
Tweet 2 data: {'user_id': 22594051, 'name': 'The Star', 'followers_count': 1903972, 'tweet_text': 'This is following the implementation of the Water Quality Conservation Programme in Dec 2024.', 'location': 'Kuala Lumpur, Malaysia', 'created_at': 'Fri Apr 18 08:25:15 +0000 2025', 'friends_count': 274, 'sentiment': 'Neutral'}
Sent to topic: MalaysiaNewsTopic
Tweet 3 data: {'user_id': 22594051, 'name': 'The Star', 'followers_count': 1903972, 'tweet_text': 'Defence Minister Ng Eng Hen said leadership succession is a cardinal strength of the party and that it was time for him to step down to make way for new candidates.', 'location': 'Kuala Lumpur, Malaysia', 'created_at': 'Fri 

# kafka consumer

In [5]:
from dataingestion.kafka_consumer import run_kafka_consumer

def main():
    """Main entry point of the application"""
    run_kafka_consumer()

if __name__ == "__main__":
    main()

#----------------------------------------#
#   Kafka Tweet Consumer Tool            #
#----------------------------------------#
Available Topics:
1. PoliticsNewsTopic
2. BusinessNewsTopic
3. SportsNewsTopic
4. EntertainmentNewsTopic
5. MalaysiaNewsTopic



Enter the number of the topic you want to consume from:  5



Output Format Options:
1. Detailed - Show each field on a separate line
2. Table - Show messages in a table format
3. Compact - Show messages in a single line each



Select output format (1-3):  1

Enter maximum number of messages to display (default: 5):  5


Consuming messages from MalaysiaNewsTopic...

Message 1:
  user_id: 22594051
  name: The Star
  followers_count: 1903972
  tweet_text: “There’s no scandal, no soap opera, no intrigue."
  location: Kuala Lumpur, Malaysia
  created_at: Fri Apr 18 08:35:13 +0000 2025
  friends_count: 274
  sentiment: Negative

Message 2:
  user_id: 22594051
  name: The Star
  followers_count: 1903972
  tweet_text: This is following the implementation of the Water Quality Conservation Programme in Dec 2024.
  location: Kuala Lumpur, Malaysia
  created_at: Fri Apr 18 08:25:15 +0000 2025
  friends_count: 274
  sentiment: Neutral

Message 3:
  user_id: 22594051
  name: The Star
  followers_count: 1903972
  tweet_text: Defence Minister Ng Eng Hen said leadership succession is a cardinal strength of the party and that it was time for him to step down to make way for new candidates.
  location: Kuala Lumpur, Malaysia
  created_at: Fri Apr 18 08:20:12 +0000 2025
  friends_count: 274
  sentiment: Positive

Message


Do you want to export these messages to CSV? (y/n):  n



Kafka consumer operation completed.


# store to HDFS

In [6]:
from dataingestion.store_to_HDFS import consume_tweets_to_hdfs

if __name__ == "__main__":
    consume_tweets_to_hdfs()



Starting to consume tweets and save to /user/hduser/raw_tweets/tweets_20250418_164452.json
Saved 10 tweets to HDFS
Saved 20 tweets to HDFS
Saved 30 tweets to HDFS
Saved 40 tweets to HDFS
Saved 50 tweets to HDFS
Saved 60 tweets to HDFS
Saved 70 tweets to HDFS
Saved 80 tweets to HDFS
Saved 90 tweets to HDFS
Saved 100 tweets to HDFS
Saved 110 tweets to HDFS
Saved 120 tweets to HDFS
Saved 130 tweets to HDFS
Saved 140 tweets to HDFS
Saved 150 tweets to HDFS
Saved 160 tweets to HDFS
Saved 170 tweets to HDFS
Saved 180 tweets to HDFS
Saved 190 tweets to HDFS
Saved 200 tweets to HDFS
Saved 210 tweets to HDFS
Saved 220 tweets to HDFS
Saved 230 tweets to HDFS
Saved 240 tweets to HDFS
Saved 250 tweets to HDFS
Saved 260 tweets to HDFS
Saved 270 tweets to HDFS
Saved 280 tweets to HDFS
Saved 290 tweets to HDFS
Saved 300 tweets to HDFS
Reached limit of 300 tweets
Consumer closed. Saved 300 tweets to HDFS.
Successfully verified file exists: /user/hduser/raw_tweets/tweets_20250418_164452.json
Run 'hadoo

In [8]:
#check availability of data
# hadoop fs -ls /user/hduser/raw_tweets/  
# hadoop fs -cat /user/hduser/raw_tweets/tweets_20250418_163452.json
